<a href="https://colab.research.google.com/github/vladimiralencar/DeepLearning-LANA/blob/master/MLP/MLP_em_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Construindo Um Algoritmo Para Rede Neural Multilayer Perceptron

### Otimização com Stochastic Gradient Descent 

Stochastic Gradient Descent (SGD) é uma versão de Gradient Descent, onde em cada passagem para a frente, obtemos um lote de dados com amostras aleatórias do conjunto de dados total. Aqui onde entra em cena o batch_size. Esse é o tamanho do lote. Idealmente, todo o conjunto de dados seria alimentado na rede neural em cada passagem para a frente, mas na prática isso acaba não sendo possível, devido a restrições de memória. SGD é uma aproximação de Gradient Descent, quanto mais lotes processados pela rede neural, melhor será a aproximação.

Uma implementação do SGD envolve:

1. Gerar lotes de dados de amostras aleatórias do conjunto de dados total.

2. Executar a rede para frente (Forward Pass) e para trás (Backward pass) para calcular o gradiente (com dados de (1)).

3. Aplicar a atualização de descida do gradiente.

4. Repitir as etapas 1-3 até a convergência ou o loop for parado por outro mecanismo (como o número de épocas, por exemplo).

Se tudo correr bem, a perda da rede vai diminuindo, indicando pesos e bias mais úteis ao longo do tempo.

In [0]:
import numpy as np

class Neuronio:
    """
    Classe base para os nós da rede.

    Argumentos:

        "nodes_entrada": Uma lista de nós com arestas para este nó.
    """
    def __init__(self, nodes_entrada = []):
        """
        O construtor do nó (é executado quando o objeto é instanciado). 
        Define propriedades que podem ser usadas por todos os nós.
        """
        # Lista de nós com arestas para este nó.
        self.nodes_entrada = nodes_entrada
        
        # Lista de nós para os quais este nó gera saída.
        self.nodes_saida = []
        
        # O valor calculado por este nó. É definido executando o método forward().
        self.valor = None
        
        # Este objeto é um dicionário com pares chaves/valor entre {} 
        # As chaves (keys) são os inputs para este nó e o valores (values) são as paciais deste nó em relação ao input.
        self.gradientes = {}
        
        # Configuramos este nó como um nó de saída para todos os nós de entrada.
        for n in nodes_entrada:
            n.nodes_saida.append(self)

    def forward(self):
        """
        Todo o nó que usar essa classe como uma classe base, precisa definir seu próprio método "forward".
        """
        raise NotImplementedError

    def backward(self):
        """
        Todo o nó que usar essa classe como uma classe base, precisa definir seu próprio método "backward".
        """
        raise NotImplementedError



class Input(Neuronio):
    """
    Input genérico para a rede.
    """
    def __init__(self):
        # O construtor da classe base deve ser executado para configurar todas as propriedades aqui.
        #
        # A propriedade mais importante de Input é valor.
        # self.valor é definido na função topological_sort().
        Neuronio.__init__(self)

    def forward(self):
        # Nada a ser feito aqui.
        pass

    def backward(self):
        # Um nó de Input não possui entradas (pois ele já é a entrada) e assim o gradiente (derivada) é zero.
        # A palavra reservada "self", é referência para este objeto.
        self.gradientes = {self: 0}
        
        # Pesos e bias podem ser inputs, assim precisamos somar o gradiente de outros gradientes de saída
        for n in self.nodes_saida:
            self.gradientes[self] += n.gradientes[self]
            

class Linear(Neuronio):
    """
    Representa um nó que realiza transformação linear.
    """
    def __init__(self, X, W, b):
        # O construtor da classe base (nó). 
        # Pesos e bias são tratados como nós de entrada (nodes_entrada).
        Neuronio.__init__(self, [X, W, b])

    def forward(self):
        """
        Executa a matemática por trás da transformação linear.
        """
        X = self.nodes_entrada[0].valor
        W = self.nodes_entrada[1].valor
        b = self.nodes_entrada[2].valor
        self.valor = np.dot(X, W) + b

    def backward(self):
        """
        Calcula o gradiente com base nos valores de saída.
        """
        # Inicializa um parcial para cada um dos nodes_entrada.
        self.gradientes = {n: np.zeros_like(n.valor) for n in self.nodes_entrada}
        
        # Ciclo através dos outputs. 
        # O gradiente mudará dependendo de cada output, assim os gradientes são somados sobre todos os outputs.
        for n in self.nodes_saida:
            
            # Obtendo parcial da perda em relação a este nó.
            grad_cost = n.gradientes[self]
            
            # Definindo o parcial da perda em relação às entradas deste nó.
            self.gradientes[self.nodes_entrada[0]] += np.dot(grad_cost, self.nodes_entrada[1].valor.T)
            
            # Definindo o parcial da perda em relação aos pesos deste nó.
            self.gradientes[self.nodes_entrada[1]] += np.dot(self.nodes_entrada[0].valor.T, grad_cost)
            
            # Definindo o parcial da perda em relação ao bias deste nó.
            self.gradientes[self.nodes_entrada[2]] += np.sum(grad_cost, axis = 0, keepdims = False)


class Sigmoid(Neuronio):
    """
    Representa o nó da função de ativação Sigmoid.
    """
    def __init__(self, node):
        # O construtor da classe base.
        Neuronio.__init__(self, [node])

    def _sigmoid(self, x):
        """
        Este método é separado do `forward` porque ele também será usado com "backward".

        `x`: Um array Numpy.
        """
        return 1. / (1. + np.exp(-x))

    def forward(self):
        """
        Executa a função _sigmoid e define a variável self.valor
        """
        input_value = self.nodes_entrada[0].valor
        self.valor = self._sigmoid(input_value)

    def backward(self):
        """
        Calcula o gradiente usando a derivada da função sigmoid 
        
        O método backward da classe Sigmoid, soma as derivadas (é uma derivada normal quando há apenas uma variável) 
        em relação à única entrada sobre todos os nós de saída.
        """
        
        # Inicializa os gradientes com zero.
        self.gradientes = {n: np.zeros_like(n.valor) for n in self.nodes_entrada}
        
        # Soma a parcial em relação ao input sobre todos os outputs.
        for n in self.nodes_saida:
            grad_cost = n.gradientes[self]
            sigmoid = self.valor
            self.gradientes[self.nodes_entrada[0]] += sigmoid * (1 - sigmoid) * grad_cost


class MSE(Neuronio):
    def __init__(self, y, a):
        """
        Função de custo para calcular o erro médio quadrático.
        Deve ser usado como último nó da rede.
        """
        # Chamada ao construtor da classe base.
        Neuronio.__init__(self, [y, a])

    def forward(self):
        """
        Calcula o erro médio ao quadrado.
        """
        # Fazemos o reshape para evitar possíveis problemas nas operações de matrizes/vetores 
        #
        # Convertendo os 2 arrays (3,1) garantimos que o resultado será (3,1) e, assim, 
        # teremos uma subtração elementwise.
        y = self.nodes_entrada[0].valor.reshape(-1, 1)
        a = self.nodes_entrada[1].valor.reshape(-1, 1)

        self.m = self.nodes_entrada[0].valor.shape[0]
        
        # Salva o output computado para o backward pass.
        self.diff = y - a
        self.valor = np.mean(self.diff**2)

    def backward(self):
        """
        Calcula o gradiente do custo.
        """
        self.gradientes[self.nodes_entrada[0]] = (2 / self.m) * self.diff
        self.gradientes[self.nodes_entrada[1]] = (-2 / self.m) * self.diff


def topological_sort(feed_dict):
    """
    Classifica os nós em ordem topológica usando o Algoritmo de Kahn.

    `Feed_dict`: um dicionário em que a chave é um nó `Input` e o valor é o respectivo feed de valor para esse nó.

    Retorna uma lista de nós ordenados.
    """

    input_nodes = [n for n in feed_dict.keys()]

    G = {}
    nodes = [n for n in input_nodes]
    while len(nodes) > 0:
        n = nodes.pop(0)
        if n not in G:
            G[n] = {'in': set(), 'out': set()}
        for m in n.nodes_saida:
            if m not in G:
                G[m] = {'in': set(), 'out': set()}
            G[n]['out'].add(m)
            G[m]['in'].add(n)
            nodes.append(m)

    L = []
    S = set(input_nodes)
    while len(S) > 0:
        n = S.pop()

        if isinstance(n, Input):
            n.valor = feed_dict[n]

        L.append(n)
        for m in n.nodes_saida:
            G[n]['out'].remove(m)
            G[m]['in'].remove(n)
            if len(G[m]['in']) == 0:
                S.add(m)
    return L


def forward_and_backward(graph):
    """
    Executa uma passagem para a frente e uma passagem para trás através de uma lista de nós ordenados.

     Argumentos:

         `Graph`: O resultado de `topological_sort`.
    """
    # Forward pass
    for n in graph:
        n.forward()

    # Backward pass
    # O valor negativo no slice permite fazer uma cópia da mesma lista na ordem inversa.
    for n in graph[::-1]:
        n.backward()


def sgd_update(params, learning_rate = 1e-2):
    """
    Atualiza o valor de cada parâmetro treinável com o SGD.

    Argumentos:

         `Trainables`: uma lista de nós `Input` que representam pesos / bias.
         `Learning_rate`: a taxa de aprendizado.
    """
    # Executa o SGD
    #
    # Loop sobre todos os parâmetros
    for t in params:
        # Alterar o valor do parâmetro, subtraindo a taxa de aprendizado 
        # multiplicado pela parte do custo em relação a esse parâmetro
        partial = t.gradientes[t]
        t.valor -= learning_rate * partial

### Executando o Grafo

http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

### Reamostragem (com reposição)

In [2]:
from sklearn.utils import resample
y = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
batch_size = 2
print(resample(y, n_samples=batch_size))
print(resample(y, n_samples=batch_size))

[15  6]
[17  7]


### Chamada da Rede Neural

In [3]:
import numpy as np
from sklearn.datasets import load_boston
from sklearn.utils import shuffle, resample

# Carrega os dados
data = load_boston()

# Variáveis de entrada e saída para treinamento supervisionado
X_ = data['data']
y_ = data['target']

# Normaliza os dados
X_ = (X_ - np.mean(X_, axis = 0)) / np.std(X_, axis = 0)

# Número de features e número de neurônios
n_features = X_.shape[1] # 13 atributos
n_hidden = 10 # numero de neuroneos

# Define valores randômicos para inicializar pesos e bias
W1_ = np.random.randn(n_features, n_hidden)
b1_ = np.zeros(n_hidden)
W2_ = np.random.randn(n_hidden, 1)
b2_ = np.zeros(1)

# Rede Neural
X, y = Input(), Input()
W1, b1 = Input(), Input()
W2, b2 = Input(), Input()

l1 = Linear(X, W1, b1)
s1 = Sigmoid(l1)
l2 = Linear(s1, W2, b2)
cost = MSE(y, l2)

# Define o feed_dict
feed_dict = {
    X: X_,
    y: y_,
    W1: W1_,
    b1: b1_,
    W2: W2_,
    b2: b2_
}

# Número de epochs (altere esse valor para ver as mudanças no resultado)
epochs = 2000

# Número total de exemplos
m = X_.shape[0]

# Batch size
batch_size = 11
steps_per_epoch = m // batch_size

# Define o grafo computacional
graph = topological_sort(feed_dict)

# Valores que serão aprendidos pela rede
params = [W1, b1, W2, b2]

# Número total de exemplos
print("Número Total de Exemplos = {}".format(m))

# Treinamento do modelo
import sys
for i in range(epochs):
    loss = 0
    for j in range(steps_per_epoch):
        
        # Passo 1 - Testa aleatoriamente um lote de exemplos
        X_batch, y_batch = resample(X_, y_, n_samples = batch_size)

        # Reset dos valores de X e y 
        X.valor = X_batch
        y.valor = y_batch

        # Passo 2 - Forward e Backpropagation
        forward_and_backward(graph)

        # Passo 3 - Otimização por SGD
        sgd_update(params)

        loss += graph[-1].valor

    sys.stdout.write("\rEpoch: {}, Custo: {:.3f}".format(i+1, loss/steps_per_epoch))

Número Total de Exemplos = 506
Epoch: 2000, Custo: 3.507

In [4]:
print(n_features)
print(len(X_))
print(len(y_))
np.set_printoptions(precision=3, suppress=True)
print(W1.valor)

13
506
506
[[ -2.198  -1.066  -0.29   -0.696   0.219   3.756  -1.594  -4.279  -0.808
    0.319]
 [  1.971  -2.936   1.301   2.514  -0.082  -0.807  -1.823  -5.18    0.572
    0.146]
 [ -1.972  -2.536   0.966   1.159   0.169   1.955  -1.461  -3.403  -0.802
    1.691]
 [  0.31   -4.984   0.025   2.239  -0.55   -0.806  -4.696   0.016   6.163
    0.142]
 [ -4.041   0.793   0.658  -3.692   1.261   2.624   2.295  -6.324  -1.177
   -0.712]
 [ -2.643  -5.224   4.95   -0.746   1.908   3.058  -0.564  -2.287   2.636
    2.281]
 [  2.39    3.252  -1.817   3.393  -1.991  -0.609  -1.648  -6.33    0.049
   -0.379]
 [ -6.36    3.207  -3.972  -8.849   1.879   5.332  -5.645  -2.248  -1.222
   -0.251]
 [ -1.668   3.478   5.235  13.111  -4.069   7.306  -1.128   3.184  -1.174
    0.063]
 [ -0.607  -5.342   1.247 -12.877   1.618  -0.809  -1.737   3.528  -3.232
    1.327]
 [ -2.546   2.9    -0.495  -7.367  -1.494   5.044   4.254  -5.833  -0.558
    0.335]
 [ -0.202   0.554  -2.971  -0.494   1.344  -0.771  -5.

In [5]:
print(b1.valor)

[  1.904  -1.641 -11.188  -6.358  -1.601   0.495  -2.175   6.44   -4.289
  -1.059]
